# Importing all the required packages and creating a connection with PostgreSQl

In [4]:
import pandas as pd
import getpass
from pandas.io import sql
import psycopg2
import sqlalchemy
import matplotlib.pyplot as plt
%matplotlib inline


In [5]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://Gauravuser:Gaurav@89@localhost:5432/InternationalDebt')

In [6]:
# Postgres username, password and database
POSTGRES_ADDRESS= '192.168.199.1'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'Gauravuser'
POSTGRES_PASSWORD = getpass.getpass('Password')
POSTGRES_DBNAME = 'InternationalDebt'

Password········


In [7]:
international_debt = pd.read_csv(r'C:\Users\sonka\Desktop\Resume\Project\Analyze International Debt Statistics\Kaggle\international_debt.csv')

In [8]:
pd.read_sql('select * from international_debt order by country_code desc limit 5',engine)

,country_name,country_code,indicator_name,indicator_code,debt
0,Zimbabwe,ZWE,"Interest payments on external debt, private no...",DT.INT.DPNG.CD,30997000.0
1,Zimbabwe,ZWE,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,71794214.8
2,Zimbabwe,ZWE,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,44396033.7
3,Zimbabwe,ZWE,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,46758660.0
4,Zimbabwe,ZWE,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,42332836.7


# 1. The World Bank's international debt data
It's not that we humans only take debts to manage our necessities. A country may also take debt to manage its economy. For example, infrastructure spending is one costly ingredient required for a country's citizens to lead comfortable lives. The World Bank is the organization that provides debt to countries.

In this notebook, we are going to analyze international debt data collected by The World Bank. The dataset contains information about the amount of debt (in USD) owed by developing countries across several categories. We are going to find the answers to questions like:

What is the total amount of debt that is owed by the countries listed in the dataset?
Which country owns the maximum amount of debt and what does that amount look like?
What is the average amount of debt owed by countries across different debt indicators?

Let's first SELECT all of the columns from the international_debt table. Also, we'll limit the output to the first ten rows to keep the output clean.


In [10]:
pd.read_sql('select * from international_debt limit 10',engine)

    

,country_name,country_code,indicator_name,indicator_code,debt
0,Afghanistan,AFG,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,72894453.7
1,Afghanistan,AFG,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,53239440.1
2,Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.9
3,Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.4
4,Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.1
5,Afghanistan,AFG,"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD,39107845.0
6,Afghanistan,AFG,"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD,23779724.3
7,Afghanistan,AFG,"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD,13335820.0
8,Afghanistan,AFG,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,100847181.9
9,Afghanistan,AFG,"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD,72894453.7


# 2. Finding the number of distinct countries
From the first ten rows, we can see the amount of debt owed by Afghanistan in the different debt indicators. But we do not know the number of different countries we have on the table. There are repetitions in the country names because a country is most likely to have debt in more than one debt indicator. 
Without a count of unique countries, we will not be able to perform our statistical analyses holistically. Here, we are going to extract the number of unique countries present in the table. 

In [13]:
pd.read_sql('SELECT  count(distinct(country_name)) as total_distinct_countries FROM international_debt',engine)

,total_distinct_countries
0,124


# 3. Finding out the distinct debt indicators
We can see there are a total of 124 countries present on the table. As we saw in the first section, there is a column called indicator_name that briefly specifies the purpose of taking the debt. Just beside that column, there is another column called indicator_code which symbolizes the category of these debts. Knowing about these various debt indicators will help us to understand the areas in which a country can possibly be indebted to.

In [16]:
pd.read_sql('select distinct(indicator_code) as distinct_debt_indicators from international_debt order by distinct_debt_indicators',engine)

,distinct_debt_indicators
0,DT.AMT.BLAT.CD
1,DT.AMT.DLXF.CD
2,DT.AMT.DPNG.CD
3,DT.AMT.MLAT.CD
4,DT.AMT.OFFT.CD
5,DT.AMT.PBND.CD
6,DT.AMT.PCBK.CD
7,DT.AMT.PROP.CD
8,DT.AMT.PRVT.CD
9,DT.DIS.BLAT.CD


# 4. Totaling the amount of debt owed by the countries
As mentioned earlier, the financial debt of a particular country represents its economic state. But if we were to project this on an overall global scale.

Let's find out the total amount of debt (in USD) that is owed by the different countries. This will give us a sense of how the overall economy of the entire world is holding up.

In [17]:
pd.read_sql('SELECT round(sum(debt)/1000000 ,2) as total_debt  FROM international_debt; ',engine)

,total_debt
0,3079734.49


# 5. Country with the highest debt
"Human beings cannot comprehend very large or very small numbers. It would be useful for us to acknowledge that fact." - Daniel Kahneman. That is more than 3 million million USD, an amount which is really hard for us to fathom.

Now that we have the exact total of the amounts of debt owed by several countries, let's now find out the country that owns the highest amount of debt along with the amount. Note that this debt is the sum of different debts owed by a country across several categories. This will help to understand more about the country in terms of its socio-economic scenarios. We can also find out the category in which the country owns its highest debt. But we will leave that for now.

In [18]:
pd.read_sql('SELECT country_name, sum(debt) as total_debt FROM international_debt GROUP BY country_name ORDER BY total_debt desc Limit 1',engine)

,country_name,total_debt
0,China,2.857935e+11


# 6. Average amount of debt across indicators
So, it was China. A more in-depth breakdown of China's debts can be found here.

We now have a brief overview of the dataset and a few of its summary statistics. We already have an idea of the different debt indicators in which the countries owe their debts. We can dig even further to find out on an average how much debt a country owes? This will give us a better sense of the distribution of the amount of debt across different indicators.

In [19]:
pd.read_sql('SELECT indicator_code AS debt_indicator,indicator_name,avg(debt) as average_debt FROM international_debt GROUP BY debt_indicator,indicator_name ORDER BY average_debt desc limit 10',engine)

,debt_indicator,indicator_name,average_debt
0,DT.AMT.DLXF.CD,"Principal repayments on external debt, long-te...",5.904868e+09
1,DT.AMT.DPNG.CD,"Principal repayments on external debt, private...",5.161194e+09
2,DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS...",2.152041e+09
3,DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1.958983e+09
4,DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1.803694e+09
5,DT.INT.DLXF.CD,"Interest payments on external debt, long-term ...",1.644024e+09
6,DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1.223139e+09
7,DT.INT.DPNG.CD,"Interest payments on external debt, private no...",1.220411e+09
8,DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1.191188e+09
9,DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1.082624e+09


# 7. The highest amount of principal repayments
We can see that the indicator DT.AMT.DLXF.CD tops the chart of average debt. This category includes repayment of long term debts. Countries take on long-term debt to acquire immediate capital. More information about this category can be found here.

An interesting observation in the above finding is that there is a huge difference in the amounts of the indicators after the second one. This indicates that the first two indicators might be the most severe categories in which the countries owe their debts.

We can investigate this a bit more so as to find out which country owes the highest amount of debt in the category of long term debts (DT.AMT.DLXF.CD). Since not all the countries suffer from the same kind of economic disturbances, this finding will allow us to understand that particular country's economic condition a bit more specifically.

In [23]:
pd.read_sql("SELECT country_name, indicator_name FROM international_debt WHERE debt = (SELECT  max(debt) FROM international_debt where indicator_code= 'DT.AMT.DLXF.CD')",engine)

,country_name,indicator_name
0,China,"Principal repayments on external debt, long-te..."


# 8. The most common debt indicator
China has the highest amount of debt in the long-term debt (DT.AMT.DLXF.CD) category. This is verified by The World Bank. It is often a good idea to verify our analyses like this since it validates that our investigations are correct.

We saw that long-term debt is the topmost category when it comes to the average amount of debt. But is it the most common indicator in which the countries owe their debt? Let's find that out.

In [24]:
pd.read_sql('select indicator_code,count(indicator_code) as indicator_count from international_debt group by indicator_code order by indicator_count desc,indicator_code desc limit 20',engine)


,indicator_code,indicator_count
0,DT.INT.OFFT.CD,124
1,DT.INT.MLAT.CD,124
2,DT.INT.DLXF.CD,124
3,DT.AMT.OFFT.CD,124
4,DT.AMT.MLAT.CD,124
5,DT.AMT.DLXF.CD,124
6,DT.DIS.DLXF.CD,123
7,DT.INT.BLAT.CD,122
8,DT.DIS.OFFT.CD,122
9,DT.AMT.BLAT.CD,122


# 9. Other viable debt issues and conclusion
There are a total of six debt indicators in which all the countries listed in our dataset have taken debt. The indicator DT.AMT.DLXF.CD is also there in the list. So, this gives us a clue that all these countries are suffering from a common economic issue. But that is not the end of the story, a part of the story rather.

Let's change tracks from debt_indicators now and focus on the amount of debt again. Let's find out the maximum amount of debt across the indicators along with the respective country names. With this, we will be in a position to identify the other plausible economic issues a country might be going through. By the end of this section, we will have found out the debt indicators in which a country owes its highest debt.

In this notebook, we took a look at debt owed by countries across the globe. We extracted a few summary statistics from the data and unraveled some interesting facts and figures. We also validated our findings to make sure the investigations are correct.

In [25]:
pd.read_sql('select country_name,indicator_code,max(debt) as maximum_debt from international_debt group by country_name,indicator_code order by maximum_debt desc limit 10',engine)

,country_name,indicator_code,maximum_debt
0,China,DT.AMT.DLXF.CD,9.621862e+10
1,Brazil,DT.AMT.DLXF.CD,9.004184e+10
2,China,DT.AMT.DPNG.CD,7.239299e+10
3,Russian Federation,DT.AMT.DLXF.CD,6.658976e+10
4,Turkey,DT.AMT.DLXF.CD,5.155503e+10
5,South Asia,DT.AMT.DLXF.CD,4.875630e+10
6,Brazil,DT.AMT.PRVT.CD,4.359870e+10
7,Russian Federation,DT.AMT.DPNG.CD,4.280015e+10
8,Brazil,DT.AMT.DPNG.CD,4.183144e+10
9,Least developed countries: UN classification,DT.DIS.DLXF.CD,4.016077e+10
